## Mikroeditering (idé)
Dette eksemplet illustrerer hvordan vi kan bruke plott til å identifisere, mikroeditere og se hvilken konsekvenser dette har for resultatet (aggregeringer)

#### Importere bibliotekene

In [ ]:
import pandas as pd

import plotly.graph_objects as go

from IPython.core.display import display
from ipywidgets import widgets

#### Lage testdata
Vi lager et datasett som inneholder følgende variabler:
- id: en enhetsidentifikator for den enkelte observasjon
- gruppe: en grupperingsvariabel som sier noe om størrelsen på verdien
- verdi: en verdi vi skal regne på

Vi har lagt inn noen feilføringer mellom gruppe og verdi, så spørsmålet er kanskje om det er *verdi* eller *gruppe* som skal rettes 


In [ ]:
df = pd.DataFrame({
    "id": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
    "gruppe": ['stor','liten','middels','middels','liten','stor','middels','middels','liten','stor','liten','middels','middels','liten','stor','middels','middels','liten'],
    "verdi": [100,2,46,30,4,90,37,1,5,80,100,54,50,8,70,48,39,6]
})

#### Funksjon som returnerer et datasett med gjennomsnitt 
fra et datasettet (df) for én verdi (col_value):
- pr gruppe (col_group)
- det totale gjennomsnittet 

In [ ]:
def calc_mean(df, col_group, col_value):
    _res_df = df.groupby(col_group).mean()[col_value]

    _total_mean = pd.Series({'totalt': df[col_value].mean()})
    
    _res_df = _res_df.append(_total_mean)

    return _res_df

#### Funksjon som kalles når brukeren klikker på en observasjon i plottet
Funksjonen gjør følgende:
- oppdaterer editeringsfeltene med verdier fra valgt observasjon
- markerer valgt observasjon rød

Funksjonen er avhengig av at følgende objekter er tilgjengelige:
- datasettet: df
- figurvariabelene: scatter og fig
- observasjonsfeltene: id_field, group_field og value_field

In [ ]:
def update_point(trace, points, selector):
    i = points.point_inds[0]
    obs = df.loc[i,:]
    id_field.value =str(obs.id)
    group_field.value =str(obs.gruppe)
    value_field.value =str(obs.verdi)

    scatter.marker.color = colors
    scatter.marker.size = [10] * 100
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = 'red'
        s[i] = 20
        with fig.batch_update():
            scatter.marker.color = c
            scatter.marker.size = s

#### Funksjon blir kalt når brukern trykker på lagre-knappen
Det vil si etter at mikroediteringen er gjennomført (lagret)
Funksjonen oppdaterer plottet:
- endrer fargen på observasjonen til grønn (fra rød)
- oppdaterer plottet (flytter observasjonen som er endret)

Funksjonen er avhengig av at følgende objekter er tilgjengelige:
- datasettet: df
- figurvariabelene: scatter og fig

In [ ]:
def update_plot(_id, _gr, _va):
    i= df.id[df.id == _id].index.tolist()[0]

    scatter.marker.color = colors
    scatter.marker.size = [10] * 100
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    c[i] = 'green'
    s[i] = 20
    with fig.batch_update():
        scatter.marker.color = c
        scatter.marker.size = s
        scatter.y = df.verdi
        scatter.x = df.gruppe

#### Funksjon som kalles når brukern trykker på lagre-knappen
Oppdaterer datasettet med endringen og kaller update_plot for å oppdatere plottet.

In [ ]:
def on_button_clicked(b):
    try:
        _id = int(id_field.value)
        _gr = group_field.value
        _va = int(value_field.value)

        df.loc[df.id == _id, ['gruppe','verdi']]= _gr, _va
        update_plot(_id, _gr, _va)
        update_mean()
    except:
       print("An exception occurred")

#### Funksjon som kalles når brukern trykker på lagre-knappen
oppdaterer verdiene for gjennomsnitt

Funksjonen er avhengig av at følgende objekter er tilgjengelige:
- datasettet: df
- gjennomsnittsfeltene: total_mean, liten_mean, middels_mean, stor_mean

In [ ]:
def update_mean(): 
    _df_mean = calc_mean(df,'gruppe','verdi').round(2)

    total_mean.value = str(_df_mean.get('totalt'))
    liten_mean.value = str(_df_mean.get('liten'))
    middels_mean.value = str(_df_mean.get('middels'))
    stor_mean.value = str(_df_mean.get('stor'))

#### Oppretter plottet som en figur (plotly.graph_objects)
Her definerer vi objekter som skal være tilgjengelig for funksjonene over:
- fig
- scatter

In [ ]:
fig = go.Figure()
fig = go.FigureWidget([go.Scatter(x=df.gruppe, y=df.verdi, mode='markers')])

scatter = fig.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
fig.layout.hovermode = 'closest'

scatter.on_click(update_point)

#### Lager tekstfelter (widgets.Text) for å presentere og endre observasjoner
Objekter som skal være tilgjengelig i andre celler:
- id_field
- group_field
- value_field

In [ ]:
id_field = widgets.Text(
    value='',
    description='Identifikator:',
    disabled=True
)

group_field = widgets.Text(
    value='',
    description='Gruppe:',
    disabled=False
)

value_field = widgets.Text(
    value='',
    description='Verdi:',
    disabled=False
)

save_button = widgets.Button(description="Save")
save_button.on_click(on_button_clicked)

#### Lager tekstfelter for gjennomsnitt

In [ ]:
df_mean = calc_mean(df,'gruppe','verdi').round(2)

total_mean = widgets.Text(
    value=str(df_mean.get('totalt')),
    description='Totalt:',
    disabled=True
)

liten_mean = widgets.Text(
    value=str(df_mean.get('liten')),
    description='Liten',
    disabled=True
)

middels_mean = widgets.Text(
    value=str(df_mean.get('middels')),
    description='Middels:',
    disabled=True
)

stor_mean = widgets.Text(
    value=str(df_mean.get('stor')),
    description='Stor:',
    disabled=True
)

#### Vise figur (plott) og tekstfelter

In [ ]:
display(total_mean,liten_mean, middels_mean, stor_mean, fig, id_field, group_field,value_field, save_button)